In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_5월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(862586, 2) Index(['split_str', 'org_idx'], dtype='object')
862586


In [3]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [4]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass

100%|██████████| 862586/862586 [17:01:02<00:00, 14.08it/s]   


In [5]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['아테네', '이외', '그리스', '국토', '전역', '문화유산', '지역', '문화', '향기', '수']
['가다', '맡다', '익히다', '들다', '하다', '느끼다', '싹트다', '하다', '이다', '하다']
['있다', '막연하다', '같다', '아니다', '이러하다', '미치다', '그렇다', '있다', '같다', '초라하다']
['에도', '는', '이', '이어서', '을', '의', '를', '는', '에서도', '부터']


In [6]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/5월/나무문화_5월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

146002


,명사,명사빈도
0,것,170335
1,수,117028
2,그,90775
3,이,86164
4,곳,72960
...,...,...
145997,묏부리,1
145998,신흥강습소,1
145999,장석환,1
146000,뭉후시선,1


In [7]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/5월/나무문화_5월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1829


,동사,동사빈도
0,하다,787273
1,되다,161818
2,보다,97473
3,않다,72703
4,되어다,70461
...,...,...
1824,한숨짓다,1
1825,주뼛거리다,1
1826,악플다다,1
1827,얽어내다,1


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/5월/나무문화_5월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1838


,형용사,형용사빈도
0,있다,382750
1,좋다,71409
2,없다,66516
3,같다,59657
4,이다,55388
...,...,...
1833,암팡지다,1
1834,여무지다,1
1835,서글서글하다,1
1836,치사스럽다,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/5월/나무문화_5월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

381


,조사,조사빈도
0,을,645109
1,의,514212
2,이,495964
3,에,466361
4,를,312327
...,...,...
376,에게다,1
377,에게야,1
378,에서까지도,1
379,에게서처럼,1


In [10]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_5월_단어종합.txt", index=False, encoding='UTF-8')

In [11]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

,split_str,org_idx,morphs,pos_tag
0,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,0,"[아테네, 이외, 에도, 그리스, 는, 국토, 전역, 이, 문화유산, 이어서, 어느...","[(아테네, Noun), (이외, Noun), (에도, Josa), (그리스, No..."
1,나는 그중에서도 학창 시절부터 익히 들었던 펠로폰네소스 Peloponnese 라는 ...,0,"[나, 는, 그, 중, 에서도, 학창, 시절, 부터, 익히, 들었던, 펠로폰네소스,...","[(나, Noun), (는, Josa), (그, Determiner), (중, No..."
2,펠로폰네소스 지역은 그리스 최초의 문화인 미케네 문명이 싹튼 지역이며 병영국가인 스...,0,"[펠로폰네소스, 지역, 은, 그리스, 최초, 의, 문화인, 미케네, 문명, 이, 싹...","[(펠로폰네소스, Noun), (지역, Noun), (은, Josa), (그리스, ..."
3,미케네는 기원전 1500년부터 기원전 1200년까지 소아시아 이집트 키프로스를 대상...,0,"[미케네, 는, 기원전, 1500년, 부터, 기원전, 1200년, 까지, 소아시아,...","[(미케네, Noun), (는, Josa), (기원전, Noun), (1500년, ..."
4,미케네인들은 매우 전투적이어서 주변 지역을 비롯해 멀리 소아시아 지역까지 점령했다고...,0,"[미케네, 인들, 은, 매우, 전투, 적, 이어서, 주변, 지역, 을, 비롯, 해,...","[(미케네, Noun), (인들, Josa), (은, Noun), (매우, Noun..."
...,...,...,...,...
862581,0 접기 펴기 멋있는 난타공연 난타 공연 너무 멋짐,19230,"[0, 접기, 펴기, 멋있는, 난타, 공연, 난타, 공연, 너무, 멋짐]","[(0, Number), (접기, Noun), (펴다, Verb), (멋있다, Ad..."
862582,와 흥이 절로 나고 신났음 ㅋ,19230,"[와, 흥이, 절로, 나고, 신났음, ㅋ]","[(오다, Verb), (흥이, Noun), (절로, Noun), (나다, Verb..."
862583,하이라이트 소리지름 주의 LED공연 하이라이트 재생 14 좋아요,19230,"[하이라이트, 소리, 지름, 주의, LED, 공연, 하이라이트, 재생, 14, 좋아요]","[(하이라이트, Noun), (소리, Noun), (지름, Noun), (주의, N..."
862584,0 접기 펴기 LED공연 하이라이트 정말 잊을수 없는 공연이었고 못 보고 갔으면 정...,19230,"[0, 접기, 펴기, LED, 공연, 하이라이트, 정말, 잊을수, 없는, 공연, 이...","[(0, Number), (접기, Noun), (펴다, Verb), (LED, Al..."


In [12]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_5월_형태소분석.txt", index=False, encoding='UTF-8')